In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis/

/content/drive/MyDrive/Notebooks/Github/twitter-sentiment-analysis


In [ ]:
import numpy as np
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm
tqdm.pandas()


In [ ]:
def prepare_tweets_df(df, stock_sym):
    tweets_dict = {
        'stock': [],
        'date': [],
        'tweets': []
    }
    for _, row in df.iterrows():
        date = row['date']
        tweets = row.iloc[2:61].values
        # print(len(tweets))
        tweets_dict['tweets'].extend(tweets)
        tweets_dict['stock'].extend([stock_sym for i in range(0, len(tweets))])
        tweets_dict['date'].extend([date for i in range(0, len(tweets))])
    return pd.DataFrame(tweets_dict)

In [ ]:
tweet_files_path = os.getcwd() + '/dataset/Tweets/'
stocks_files_path = os.getcwd() + '/dataset/Stocks/'

stock_names = ['AAPL', 'AMT', 'AMZN', 'APRN', 'C', 'DIS', 'FB', 'GE', 'GOOGL', 'HTZ', 'LUV', 'MSFT', 'NFLX', 'SBUX', 'TSLA']

tweets_df_list = []
for stock_name in stock_names:
    tweets_df_list.append(prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name))

tweets_df = pd.concat(tweets_df_list, axis = 0)

# tweets_df = pd.concat([prepare_tweets_df(pd.read_csv(tweet_files_path + stock_name + '.csv'), stock_name) for stock_name in stock_names], axis = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
tweets_df.sample(20)
print(tweets_df.shape)
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
# print(tweets_df.duplicated().sum())
print(len(tweets_df)-len(tweets_df.drop_duplicates()))
tweets_df = tweets_df.drop_duplicates()
print(tweets_df.isna().sum())
print(tweets_df.shape)

(1269444, 3)
stock           0
date            0
tweets    1002313
dtype: int64
17936
stock     0
date      0
tweets    0
dtype: int64
(249195, 3)


In [ ]:
# Due to incorrect formatting of the source csv files, some strings are available in date column. 
# So we are forcing those values to be NaT
# Also dropping those rows
tweets_df['date'] = pd.to_datetime(tweets_df['date'], errors='coerce')
print(tweets_df.isna().sum())
tweets_df = tweets_df.dropna()
print(tweets_df.shape)
tweets_df = tweets_df.reset_index(drop = True)

stock       0
date      170
tweets      0
dtype: int64
(249025, 3)


In [ ]:
tweets_df.to_csv('./dataset/Tweets/tweets.csv', index = False)

In [ ]:
# !pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 85.9 MB/s 
     |████████████████████████████████| 163 kB 93.9 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 127 kB 91.4 MB/s 
     |████████████████████████████████| 115 kB 93.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='./dataset/Tweets/tweets.csv', split='train')
dataset[100]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c48266b84ebf5b21/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


{'stock': 'AAPL',
 'date': '2019-07-16',
 'tweets': "Maps: What's New in iOS 13  http://dlvr.it/R8XYGN\xa0 $AAPL  pic.twitter.com/t2NPYX6kkE"}

In [ ]:
# !pip install -q -U transformers
# !pip install -U torch torchvision torchaudio

     |████████████████████████████████| 5.3 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 70.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████  | 834.1 MB 115.7 MB/s eta 0:00:01tcmalloc: large alloc 1147494400 bytes == 0x65c36000 @  0x7f23848d4615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 890.2 MB 6.8 kB/s 
     |████████████████████████████████| 24.3 MB 61.4 MB/s 
     |████████████████████████████████| 4.2 MB 55.4 MB/s 
     |████████████████████████████████| 849 kB 83.1 MB/s 
     |████████████████████████████████| 21.0 MB 5.9 MB/s 
     |████████████████████████████████| 317.1 MB 29 kB/s 
     |██████████████████

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 557.1 MB 10 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.13.0 which is incompatible.


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Preprocess text (username and link placeholders)
# def preprocess(text):
#     new_text = []
#     for t in text.split(" "):
#         t = '@user' if t.startswith('@') and len(t) > 1 else t
#         t = 'http' if t.startswith('http') else t
#         new_text.append(t)
#     return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
# config = AutoConfig.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL, truncation=True, padding = 'max_length', max_length = 512)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.to(device)
device


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device(type='cuda')

In [ ]:
def preprocess_dataset(row):
  text = row['tweets']
  new_text = []
  for t in text.split(" "):
      t = '@user' if t.startswith('@') and len(t) > 1 else t
      t = 'http' if t.startswith('http') else t
      new_text.append(t)
  row['tweets'] = " ".join(new_text)
  return row

In [ ]:
preprocessed_dataset =  dataset.map(preprocess_dataset)

In [ ]:
def tokenize_function(row):
  return tokenizer(row['tweets'], return_tensors='pt', truncation=True, padding = 'max_length', max_length = 512).to(device)

In [ ]:
tokenized_datasets = preprocessed_dataset.map(tokenize_function, batched=True)

  0%|          | 0/250 [00:00<?, ?ba/s]

In [ ]:
len(tokenized_datasets)

249025

In [ ]:
# I am changing the type of tensor columns (input_ids, attention_mask) to torch type
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')


In [ ]:
dataloader = torch.utils.data.DataLoader(tokenized_datasets, batch_size=32)

In [ ]:
scores_dict = {
    'negative': [],
    'neutral': [],
    'positive': []
}

for batch in tqdm(dataloader):
  # with torch.no_grad():        
  outputs = model(**batch)
  logits = outputs.logits
  logits = logits.detach().cpu().numpy()
  for logit in logits:
    scores = softmax(logit)
    scores_dict['negative'].append(scores[0])
    scores_dict['neutral'].append(scores[1])
    scores_dict['positive'].append(scores[2])


  0%|          | 0/7783 [00:00<?, ?it/s]

In [ ]:
scores_df = pd.DataFrame(scores_dict)
print(scores_df.shape)
display(scores_df.head())

(249025, 3)


,negative,neutral,positive
0,0.004799,0.442696,0.552505
1,0.016513,0.858550,0.124938
2,0.004786,0.595696,0.399518
3,0.010387,0.797119,0.192494
4,0.005847,0.707267,0.286886


In [ ]:
len(tweets_df)

249025

In [ ]:
final_df = pd.concat([tweets_df, scores_df], axis = 1)
final_df.to_csv('./dataset/Tweets/tweets-sentiment.csv', index = False)